In [1]:
import os
os.chdir("../")

In [2]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel
from transformers import BartForConditionalGeneration

In [ ]:
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
test_dataset = dataset["test"]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [ ]:
test_dataset

In [ ]:
import pandas as pd

In [ ]:
def aggregation_analysis(index):

    question = test_dataset[index]["question"]
    # question = "How many people in total were murdered in 1940/41 and 1941/42?"
    # question = "what was the total number of people who attented on november 17 and november 23"
    table = test_dataset[index]["table"]

    table_column_names = test_dataset[index]["table"]["header"]
    table_content_values = test_dataset[index]["table"]["rows"]

    answer = test_dataset[index]["answers"]
    answer_list = answers = [str(a).lower() for a in test_dataset[index]["answers"]]
    answer = f" {tokenizer.special_tokens_map['sep_token']} ".join(answer)
    # answer = "216,000"
    # answer = "1015"

    table = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})
    print(question)
    display(table)
    print(answer)

    return


    input = tokenizer(table, question, add_special_tokens = True, padding = True, truncation = True, max_length = 960, return_tensors = "pt",
                        return_token_type_ids = True, return_attention_mask = True)
    
    answer_pred_ids = model.generate(inputs = input["input_ids"], attention_mask = input["attention_mask"], max_new_tokens = 64, num_beams = 3, early_stopping = True)
    answer_pred = tokenizer.batch_decode(answer_pred_ids, skip_special_tokens = True)

    display(table)
    table = table[["day", "attendance"]].iloc[[1, 2]]
    display(table)

    input = tokenizer(table, question, add_special_tokens = True, padding = True, truncation = True, max_length = 960, return_tensors = "pt",
                        return_token_type_ids = True, return_attention_mask = True)
    
    answer_pred_ids_decomposed = model.generate(inputs = input["input_ids"], attention_mask = input["attention_mask"], max_new_tokens = 64, num_beams = 3, early_stopping = True)
    answer_pred_decomposed = tokenizer.batch_decode(answer_pred_ids, skip_special_tokens = True)

    table["attendance"][1] = "3 hundreds 6 tens 3 units"
    table["attendance"][2] = "6 hundreds 5 tens 2 units"

    input = tokenizer(table, question, add_special_tokens = True, padding = True, truncation = True, max_length = 960, return_tensors = "pt",
                        return_token_type_ids = True, return_attention_mask = True)
    
    # print(tokenizer.batch_decode(input["input_ids"], skip_special_tokens = True))
    answer_pred_ids_verbalised = model.generate(inputs = input["input_ids"], attention_mask = input["attention_mask"], max_new_tokens = 64, num_beams = 3, early_stopping = True)
    answer_pred_verbalised = tokenizer.batch_decode(answer_pred_ids, skip_special_tokens = True)

    print(question)
    display(table)
    print(answer)
    print(answer_pred)
    print(answer_pred_decomposed)
    print(answer_pred_verbalised)

In [ ]:
aggregation_analysis(1)

In [ ]:
aggregation_analysis(7)

In [ ]:
aggregation_analysis(13)